In [64]:
# ----------------------------
# Prepare training data from Metadata file
# ----------------------------
import pandas as pd
from pathlib import Path

data_path = r'D:\0_Workplace\Project_Python\audio_process\data\train'

# Read metadata file
metadata_file = r"D:\0_Workplace\Project_Python\audio_process\data\train\audio_labels2.csv"
df = pd.read_csv(metadata_file)
df.head()

# Construct file path by concatenating fold and file name
df['relative_path'] = '/'+df['Label'].astype(str) + '/' + df['Filename'].astype(str)

# Take relevant columns
# df = df[['relative_path', 'classID']]
df.head()

,Filename,Label,classID,relative_path
0,enoutput_02024_10_07_22_49_51_552246.wav,fold_en,2,/fold_en/enoutput_02024_10_07_22_49_51_552246.wav
1,enoutput_10002024_10_07_22_49_51_552246.wav,fold_en,2,/fold_en/enoutput_10002024_10_07_22_49_51_5522...
2,enoutput_1002024_10_07_22_49_51_552246.wav,fold_en,2,/fold_en/enoutput_1002024_10_07_22_49_51_55224...
3,enoutput_10052024_10_07_22_49_51_552246.wav,fold_en,2,/fold_en/enoutput_10052024_10_07_22_49_51_5522...
4,enoutput_10102024_10_07_22_49_51_552246.wav,fold_en,2,/fold_en/enoutput_10102024_10_07_22_49_51_5522...


In [65]:
from torch.utils.data import random_split
from dataset import SoundDS
import torch

myds = SoundDS(df, data_path)

# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)

In [66]:
from torch import nn
from audioclassifier import AudioClassifier
# Create the model and put it on the GPU if available
model = nn.DataParallel(AudioClassifier())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Check that it is on Cuda
next(model.parameters()).device

device(type='cpu')

In [73]:
# ----------------------------
# Training Loop
# ----------------------------
import torch.utils
import torch.utils.tensorboard


def training(model, train_dl, num_epochs):
    # Tensorboard
    writer = torch.utils.tensorboard.SummaryWriter()
    # Loss Function, Optimizer and Scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')
    # Repeat for each epoch
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_prediction = 0
        total_prediction = 0

        # Repeat for each batch in the training set
        for i, data in enumerate(train_dl):
            # Get the input features and target labels, and put them on the GPU
            print(data)
            inputs = data[0].to(device)
            labels = data[2].to(device)

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Keep stats for Loss and Accuracy
            running_loss += loss.item()

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs,1)
            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

            #if i % 10 == 0:    # print every 10 mini-batches
            #    print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
        
        # Print stats at the end of the epoch
        num_batches = len(train_dl)
        avg_loss = running_loss / num_batches
        avg_acc = correct_prediction/total_prediction
        writer.add_scalar("Loss/train", avg_loss, epoch)
        writer.add_scalar("Acc/train", avg_acc, epoch)
        print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {avg_acc:.2f}')
        # Save model
        torch.save(model.state_dict(), 'model.pt')
    print('Finished Training')
    

In [74]:
num_epochs=2
training(model, train_dl, num_epochs)

[tensor([[[[ 20.9928,  -3.7563, -11.4807,  ...,  16.5229,  11.1248,  10.3414],
          [ 21.4776,  -1.2614,  -4.3272,  ...,  12.3113,   2.1380,   3.9434],
          [ 26.0693,  15.7920,  19.4872,  ...,   5.6844, -12.8872,   6.8701],
          ...,
          [-39.5577, -39.5577, -37.6804,  ..., -39.5577, -39.5577, -39.5577],
          [-39.5577, -39.5577, -39.5577,  ..., -39.5577, -39.5577, -39.5577],
          [-39.5577, -39.5577, -39.5577,  ..., -39.5577, -39.5577, -39.5577]],

         [[ 20.9928,  -3.7563, -11.4807,  ...,  16.5229,  11.1248,  10.3414],
          [ 21.4776,  -1.2614,  -4.3272,  ...,  12.3113,   2.1380,   3.9434],
          [ 26.0693,  15.7920,  19.4872,  ...,   5.6844, -12.8872,   6.8701],
          ...,
          [-39.5577, -39.5577, -37.6804,  ..., -39.5577, -39.5577, -39.5577],
          [-39.5577, -39.5577, -39.5577,  ..., -39.5577, -39.5577, -39.5577],
          [-39.5577, -39.5577, -39.5577,  ..., -39.5577, -39.5577, -39.5577]]],


        [[[ 11.6899,  -0.17

IndexError: list index out of range